![xwidgets](../src/xwidgets.svg)

<center> <h1>Widgets in C++ with xeus and cling</h1> </center>

## Button widget

In [ ]:
#include <iostream>
#include "xwidgets/xbutton.hpp"

In [ ]:
xw::button button;

In [ ]:
void foo()
{
    button.description = "button";
}

In [ ]:
button.on_click(foo);
    
button.display()

## Numerical input

In [ ]:
#include "xwidgets/xnumeral.hpp"

In [ ]:
xw::numeral<double> numeral;
numeral.display()

In [ ]:
numeral.value = 4;

In [ ]:
numeral.value()

## Image

In [ ]:
#include "xwidgets/ximage.hpp"
#include "xwidgets/xprogress.hpp"

In [ ]:
auto input = xw::image_from_file("my_awesome_cat.jpg").finalize();
input

![xtensor-fftw](../src/xtensor-fftw.svg)

In [ ]:
#pragma cling(optimize, 3)

#include <string>
#include <fstream>
#include <complex>

#include "xtl/xbase64.hpp"

#include "xtensor/xarray.hpp"
#include "xtensor/xview.hpp"
#include "xtensor/xbuilder.hpp"

#include "xtensor-io/ximage.hpp"

#include "xtensor-fftw/basic.hpp"
#include "xtensor-fftw/helper.hpp"

In [ ]:
std::vector<char> read_file(const char* filename)
{
    std::basic_ifstream<char> file(filename, std::ios::binary);
    return std::vector<char>((std::istreambuf_iterator<char>(file)), std::istreambuf_iterator<char>());
}

In [ ]:
template <class E>
std::vector<char> to_png_buffer(const xt::xexpression<E>& e)
{
    const char* temp_filename = "/tmp/xio_image.png";
    xt::dump_image(temp_filename, e);
    return read_file(temp_filename);
}

In [ ]:
xw::progress<double> progress;
progress.description = "Progress:";
progress

In [ ]:
auto output = xw::image();

xt::xarray<double> image_R;
xt::xarray< std::complex<double> > d_image_dx_fs_bw;
xt::xarray< std::complex<double> > d_image_dy_fs_bw;

image_R = xt::view(xt::load_image("my_awesome_cat.jpg"), xt::all(), xt::all(), 0);
progress.value = 10;

auto image_fs_bw = xt::fftw::rfft2(image_R);
progress.value = 20;

std::complex<double> i {0, 1};
d_image_dx_fs_bw = i * xt::view(xt::fftw::fftscale<double>(image_R.shape()[0]), xt::all(), xt::newaxis()) * image_fs_bw;
progress.value = 30;
d_image_dy_fs_bw = i * xt::fftw::rfftscale<double>(image_R.shape()[1]) * image_fs_bw;
progress.value = 40;

auto d_image_dx_bw = xt::fftw::irfft2(d_image_dx_fs_bw);
progress.value = 50;
auto d_image_dy_bw = xt::fftw::irfft2(d_image_dy_fs_bw);
progress.value = 60;

auto d_image_grad_bw = xt::sqrt(d_image_dx_bw * d_image_dx_bw + d_image_dy_bw * d_image_dy_bw);
progress.value = 70;

double amax_d_image_grad_bw3 = xt::amax(d_image_grad_bw)[0];
progress.value = 80;

auto res = xt::cast<unsigned char>(255 - (d_image_grad_bw / amax_d_image_grad_bw3 * 255));
progress.value = 90;
output.value = to_png_buffer(res);
progress.value = 100;

output

![xleaflet](../src/xleaflet.svg)

<center> <h1>C++ backend for the jupyter-leaflet map visualization library</h1> </center>

In [ ]:
#include <random>
#include <array>
#include <vector>
#include <chrono>
#include <thread>

#include "xwidgets/xlink.hpp"
#include "xwidgets/xslider.hpp"

#include "xleaflet/xmap.hpp"
#include "xleaflet/xbasemaps.hpp"
#include "xleaflet/xmarker.hpp"
#include "xleaflet/xheatmap.hpp"

# Base example with xwidgets

In [ ]:
std::array<double, 2> center = {52.204793, 360.121558};

auto map1 = xlf::map_generator()
    .center(center)
    .zoom(9)
    .finalize();
map1.display();

## Link the zoom level to a slider widget

In [ ]:
auto slider = xw::slider_generator<int>()
    .max(map1.max_zoom)
    .min(map1.min_zoom)
    .description("zoom:")
    .value(map1.zoom)
    .finalize();

auto link = xw::link(slider, "value", map1, "zoom");

slider

## Add a Strava layer

In [ ]:
auto strava = xlf::basemap({"Strava", "All"});

map1.add_layer(strava);

## Add a marker

In [ ]:
auto marker = xlf::marker_generator()
    .location({52.1, 359.9})
    .finalize();

map1.add_layer(marker);

# Heatmap layer

In [ ]:
auto map2 = xlf::map_generator()
    .center({37.58, 261.65})
    .zoom(5)
    .finalize();
map2

In [ ]:
std::random_device rd;
std::mt19937 mt(rd());
std::uniform_real_distribution<double> rd_latitude(34.0, 40.0);
std::uniform_real_distribution<double> rd_longitude(255.0, 265.0);
std::uniform_real_distribution<double> rd_intensity(0.0, 1000.0);

std::vector<std::array<double, 3>> heatmap_points;

for (std::size_t i = 0; i < 100; ++i)
{
    heatmap_points.push_back({rd_latitude(mt), rd_longitude(mt), rd_intensity(mt)});
}

In [ ]:
auto heatmap = xlf::heatmap_generator()
    .locations(heatmap_points)
    .finalize();
map2.add_layer(heatmap);

In [ ]:
for (std::size_t i = 0; i < 10; ++i)
{
    std::this_thread::sleep_for(std::chrono::seconds(1));
    heatmap.radius = heatmap.radius() + 1;
}

## Velocity layer

In [ ]:
#include <fstream>
#include "nlohmann/json.hpp"
#include "xleaflet/xvelocity.hpp"

In [ ]:
auto map3 = xlf::map_generator()
    .center({37.58, 261.65})
    .zoom(5)
    .finalize();

auto base_layer = xlf::basemap({"CartoDB", "DarkMatter"});
map3.add_layer(base_layer);

map3

In [ ]:
std::ifstream file("velocity_data.json");
nlohmann::json data;
file >> data;

auto velocity = xlf::velocity_generator()
    .data(data)
    .velocity_scale(0.01)
    .max_velocity(20)
    .display_options(R"({
        "velocityType": "Global Wind",
        "displayPosition": "bottomleft",
        "displayEmptyString": "No wind data"
    })")
    .finalize();

map3.add_layer(velocity);